In [1]:
import pandas as pd
import numpy as np
import wrds
import nasdaqdatalink as ndl
from tqdm import tqdm
import os
try:
    os.chdir('../')
except:
    pass

import keys

# 0. S&P500 Ticker List

https://github.com/fja05680/sp500

- ./data/sp500_tickers/sp500.csv : current sp500 components
- ./data/sp500_tickers/sp500_historical.csv : tickers from 1996 to 2023

In [2]:
os.getcwd()

'/Users/hun/PersonalGit/Uchicago_Programming/ttic-unsupervised/Unsupervised_Final_Project'

In [21]:
tickers = pd.read_csv("./data/sp500_tickers/sp500_historical.csv", parse_dates=['date']).set_index('date')
tickers.shape

# sample = tuple(tickers[tickers.date == '2006-01-03'].tickers.iloc[0].split(","))
# len(sample)

ticker_set = set()
for i in tickers.loc["2006":"2019"].tickers:
    set2 = set(i.split(","))
    ticker_set = ticker_set.union(set2)

sample = tuple(ticker_set)
len(sample)

835

In [3]:
# with open("tickers.txt", "w") as f:
#     for ticker in sample:
#         f.write(ticker + "\n")

# 1. WRDS

In [7]:
conn = wrds.Connection(wrds_username = "younghunlee")

Loading library list...
Done


## 1. Price

In [ ]:
price_query = f"""SELECT * 
    FROM wrdsapps_finratio.firm_ratio 
    WHERE ticker IN {sample[:5]}
    AND adate >= '2010-01-01' AND adate <= '2019-12-31'
    ORDER BY adate ASC;"""

In [29]:
query = f"""SELECT * 
    FROM wrdsapps_finratio.firm_ratio 
    WHERE ticker IN ('ABT')
    AND public_date >= '1999-06-01' AND public_date <= '2003-12-31'
    ORDER BY public_date ASC;"""

- public_date : market available date
- gsesector : GICS industry portfolio value
- gsedecs: GICS sector name

In [32]:
temp = conn.raw_sql(query)
temp[:10]

,gvkey,permno,adate,qdate,public_date,capei,be,bm,evm,pe_op_basic,...,ffi30_desc,ffi30,ffi38_desc,ffi38,ffi48_desc,ffi48,ffi49_desc,ffi49,ticker,cusip
0,001078,20482.0,1998-12-31,1999-03-31,1999-06-30,36.620610,6062.238,0.085235,17.696793,28.718354,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410
1,001078,20482.0,1998-12-31,1999-03-31,1999-07-31,34.653387,6062.238,0.085235,17.696793,27.175633,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410
2,001078,20482.0,1998-12-31,1999-06-30,1999-08-31,34.103282,6439.271,0.093293,17.836545,26.774691,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410
3,001078,20482.0,1998-12-31,1999-06-30,1999-09-30,28.845283,6439.271,0.093293,17.836545,22.646605,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410
4,001078,20482.0,1998-12-31,1999-06-30,1999-10-31,31.744554,6439.271,0.093293,17.836545,24.922840,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410
5,001078,20482.0,1998-12-31,1999-09-30,1999-11-30,29.487998,6669.653,0.119451,17.641371,24.203822,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410
6,001078,20482.0,1998-12-31,1999-09-30,1999-12-31,28.178498,6669.653,0.119451,17.641371,23.128981,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410
7,001078,20482.0,1998-12-31,1999-09-30,2000-01-31,25.268498,6669.653,0.119451,17.641371,20.740446,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410
8,001078,20482.0,1999-12-31,1999-12-31,2000-02-29,24.286846,7451.374,0.132643,14.035345,19.760479,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410
9,001078,20482.0,1999-12-31,1999-12-31,2000-03-31,26.071677,7451.374,0.132643,14.035345,21.070359,...,HLTH,8.0,CHEMS,14.0,DRUGS,13.0,DRUGS,13.0,ABT,00282410


In [33]:
# res = pd.DataFrame( )
init = True
for y in tqdm(range(1999, 2020)):
    query = f"""SELECT * 
        FROM wrdsapps_finratio.firm_ratio 
        WHERE ticker IN {sample}
        AND public_date >= '{y}-01-01' AND public_date <= '{y}-12-31'
        ORDER BY public_date ASC;"""
    temp = conn.raw_sql(query)
    if init:
        temp.to_csv(f"./data/fin_firm_raw2.csv", index = False)
        init = False
    else:
        temp.to_csv(f"./data/fin_firm_raw2.csv", mode='a', header=False, index = False)

100%|██████████| 21/21 [00:55<00:00,  2.63s/it]


# 2. NasdaqDataLink

In [26]:
try:
    zacks_tickers = pd.read_csv('./data/zacks_tickers.csv', dtype={tickers:str})
except:
    zacks_tickers = ndl.get_table('ZACKS/MT', paginate = True)
    # zacks_tickers = zacks_tickers[(zacks_tickers.active_ticker_flag == 'N') & (zacks_tickers.country_code == 'US')]
    # zacks_tickers = zacks_tickers[(zacks_tickers.active_ticker_flag == 'N') & (zacks_tickers.country_code == 'US')]
    # zacks_tickers = zacks_tickers[~zacks_tickers.zacks_x_sector_desc.isin(["Auto/Tires/Trucks", "Finance"])]
    zacks_tickers.to_csv('./data/zacks_tickers.csv')
# ticker_avail = ','.join(zacks_tickers.ticker.unique())

ticks = list(zacks_tickers.ticker)
in_zacks = []
for s in sample:
    if s in ticks:
        in_zacks.append(s)

ticker_avail = ','.join(in_zacks)

In [27]:
len(in_zacks)

772

In [4]:
try:
    prices = pd.read_csv('../data/daily_prices.csv')
    prices.date = pd.to_datetime(prices.date)
    prices = prices.set_index(['ticker','date']).sort_index()[['close', 'adj_close','adj_volume']]
except:
    ndl.export_table('QUOTEMEDIA/PRICES',
                        ticker = list(sample),
                        date = {'gte': "2000-01-01", 'lte': "2019-12-31" }, 
                        filename = './data/daily_prices.zip')

In [42]:
import os

In [ ]:
start_date = '2006-01-01'
end_date = '2020-01-31'

try:
    raise Exception
    fr = pd.read_csv("./data/zacks/fr.csv")
    fc = pd.read_csv("./data/zacks/fc.csv")
    mkval = pd.read_csv("./data/zacks/mktv.csv")
    shrs = pd.read_csv("./data/zacks/shrs.csv")
except:
    ndl.export_table('ZACKS/FR', ticker = ticker_avail, per_end_date = {'gte': start_date, 'lte': end_date }, filename = './data/zacks/ZACKS_FR_test.zip')
    ndl.export_table('ZACKS/FC', ticker = ticker_avail, per_end_date = {'gte': start_date, 'lte': end_date }, filename = './data/zacks/ZACKS_FC.zip')
    ndl.export_table('ZACKS/MKTV', ticker = ticker_avail, per_end_date = {'gte': start_date, 'lte': end_date }, filename = './data/zacks/ZACKS_mktv.zip')
    ndl.export_table('ZACKS/SHRS', ticker = ticker_avail, per_end_date = {'gte': start_date, 'lte': end_date }, filename = './data/zacks/ZACKS_shares.zip')

In [7]:
start_date = '2000-01-01'
end_date = '2010-01-31'

try:
    raise Exception
    fr = pd.read_csv("./data/zacks/fr.csv")
    fc = pd.read_csv("./data/zacks/fc.csv")
    mkval = pd.read_csv("./data/zacks/mktv.csv")
    shrs = pd.read_csv("./data/zacks/shrs.csv")
except:
    ndl.export_table('ZACKS/FR', ticker = ticker_avail, per_end_date = {'gte': start_date, 'lte': end_date }, filename = './data/zacks/ZACKS_FR.zip')
    # ndl.export_table('ZACKS/FC', ticker = ticker_avail, per_end_date = {'gte': start_date, 'lte': end_date }, filename = './data/zacks/ZACKS_FC.zip')
    # ndl.export_table('ZACKS/MKTV', ticker = ticker_avail, per_end_date = {'gte': start_date, 'lte': end_date }, filename = './data/zacks/ZACKS_mktv.zip')
    # ndl.export_table('ZACKS/SHRS', ticker = ticker_avail, per_end_date = {'gte': start_date, 'lte': end_date }, filename = './data/zacks/ZACKS_shares.zip')